## Указание конфигурации программы

In [ ]:
input_type = 'v' # or graph file, or simply text file with embedding
model_type_delay = '///' # the one we are gonna use or every one of them for delay
model_type_area = '///' # the one we are gonna use or every one of them for area
output_data = 'd' # or the 'a' or 'a_d'
path = '///' # path of the file we are gonna use our program
#optimization_type = 'abcStatsBalance' # or just abcStats or abcStatsResyn2 
size = '-10' # size of our scheme on which our model depends(can be 10-100 or 100-)
#here we will save the filenames for the models of different sizes
models1 = ['CatBoost', 'RandomForestRegressor', 'LGBMRegressor', 'NeuralNetwork']
models2 = ['CatBoost', 'RandomForestRegressor', 'LGBMRegressor', 'NeuralNetwork']
models3 = ['CatBoost', 'RandomForestRegressor', 'LGBMRegressor', 'NeuralNetwork']

## Основная часть

In [ ]:
from node2vec import Node2Vec
import numpy as np
import os
import pandas as pd
import ast
import networkx as nx
def parser(file_, verbose=0):
    GATES = ['not', 'and', 'or', 'nand', 'nor', 'xor', 'xnor', 'buf', 'assign']
    # all input ports are taken as nodes to visualize better
    input_nodes = []
    # all output ports are taken as nodes to visualize better
    output_nodes = []
    # nodes
    gates = []
    # [edges][(tail, head)] here the first index is the name of the wire;
    # tail and head are the nodes to which the edge is connected to
    wires = []

    v_file = open(file_, 'r')
    n = 0
    line = v_file.readline()
    while line:
        # remove all leading spaces
        line = line.lstrip()

        if line.startswith('//') or line.startswith('module'):
            pass
        elif line.startswith('input'):
            # find all the input ports
            line = line.lstrip('input').split(sep=',')[:-1]
            line = [x.strip(',;\n ') for x in line]
            input_nodes.extend(line)
        elif line.startswith('output'):
            # find all the output ports
            line = line.lstrip('output').split(sep=',')
            line = [x.strip(',;\n ') for x in line]
            output_nodes.extend(line)
        elif line.startswith('assign'):
            gate = 'assign' + str(n)
            n += 1
            gates.append(gate)

            line = ''.join(line.split(' ')[1:])[:-1]
            line = (line[:-1]).split('=')

            _output = line[0]
            _input = line[1:]

            if _output in output_nodes:
                wires.append([_output, gate, [_output]])
            else:
                flag = False
                for i in wires:
                    if i[0] == _output:
                        i[1] = gate
                        flag = True
                        break
                if not flag:
                    wires.append([_output, gate, []])

            for i in _input:
                # print(i, end='')
                if i in input_nodes:
                    flag = False
                    for j in wires:
                        if j[0] == i:
                            j[2].append(gate)
                            flag = True
                            break
                    if not flag:
                        wires.append([i, i, [gate]])
                else:
                    flag = False
                    for j in wires:
                        if j[0] == i:
                            j[2].append(gate)
                            flag = True
                            break
                    if not flag:
                        wires.append([i, ' ', [gate]])
        elif any(g in line for g in GATES):
            # find all the connections and gates

            # remove gate type (gate instantiation) from line
            stri = ''
            line = (line.split(sep=' ', maxsplit=1))
            g = line[0]
            line = line[1:]
            line = stri.join(line)

            # extract gate name from line, append to gates and remove from line
            line = line.split('(')
            gate = g + str(n) + (line[0]).strip(',;\n ()')
            n = n + 1
            gates.append(gate)
            line = line[1:]

            # extract input and output ports of gate and append wires
            stri = ''
            line = stri.join(line)
            line = [x.strip(',;\n ()') for x in line.split(',')]
            _output = line[0]
            _input = line[1:]


            # print(_output, _input)
            # checking if a wire exists in wires with name _output and adding its edge parameters
            if _output in output_nodes:
                wires.append([_output, gate, [_output]])
            else:
                flag = 0
                for i in wires:
                    if(i[0] == _output):
                        i[1] = gate
                        flag = 1
                        break

                if flag == 0:
                    wires.append([_output, gate, []])

            for i in _input:
                # print(i, end='')
                if i in input_nodes:
                    flag = 0
                    for j in wires:
                        if j[0] == i:
                            j[2].append(gate)
                            flag = 1
                            break
                    if flag == 0:
                        wires.append([i, i, [gate]])

                else:
                    flag = 0
                    for j in wires:
                        if(j[0] == i):
                            j[2].append(gate)
                            flag = 1
                            break

                    if flag == 0:
                        wires.append([i, ' ', [gate]])

        line = v_file.readline()

    return input_nodes, output_nodes, gates, wires

def grapher(in_n, out_n, nodes, edges):

    # in_n, out_n, nodes, edges = verilog_parser.parser(file_)

    G=nx.DiGraph()
    G.add_nodes_from(in_n)
    G.add_nodes_from(out_n)
    G.add_nodes_from(nodes)


    for i in edges:
        for j in i[2]:
            G.add_edge(i[1], j)

    return G

def main(path):
    in_n, out_n, nodes, edges = parser(path) # файл на verilog нужно подгрузить в коллаб
    G = grapher(in_n, out_n, nodes, edges)
    return(G)

tensor = []

c = path
if os.path.isfile(c) and os.path.getsize(c) != 0:
    matr = []
    graph = main(c)
    node2vec = Node2Vec(graph, dimensions=25, walk_length=3, num_walks=20, workers=4)
    model = node2vec.fit(window=4, min_count=1, batch_words=4)
    for j in range(len(model.wv)):
        matr.append(model.wv[model.wv.index_to_key[j]].tolist())
    tensor.append(matr)
else:
    tensor.append(None)

data = [[1, 1, 1]]
 
df = pd.DataFrame(data, columns=['File', 'Area', 'Delay'])

final = df.assign(embedding = tensor)

In [ ]:
data = final
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [ ]:
new_embedding = []
for str in data['embedding']:
    result = []
    if '\n' in str:
        res = str.split('\n')
    else:
        res = str.split('], ')
    for mass in res:
        s = mass.replace(', dtype=float32',' ').replace('(', ' ').replace(')',' ').replace(',',' ').replace('[','').replace(']','').split(' ')
        tmp = []
        for word in s:
            if is_number(word):
                tmp.append(float(word))
        result.append(tmp)
    r = [np.array(sublist) for sublist in result]
    new_embedding.append(r)

In [ ]:
import pickle

# Загрузка значения из файла
if size == '-10':
        file_path = 'max_subarrays.pickle1'
    elif size == '10-100':
        file_path = 'max_subarrays.pickle2'
    else:
        file_path = 'max_subarrays.pickle3'
with open(file_path, 'rb') as f:
    max_subarrays = pickle.load(f)

In [ ]:
for i in range(len(new_embedding)):
    while len(new_embedding[i]) < max_subarrays:
        new_embedding[i] = np.vstack([new_embedding[i], np.zeros(25)])

In [ ]:
for i in range(len(new_embedding)):
    tmp = []
    for j in range (len(new_embedding[i])):
        for k in new_embedding[i][j]:
            tmp.append(k)
    new_embedding[i] = np.array(tmp)

In [ ]:
s = pd.Series(new_embedding)
data['New_embeddings'] = s

In [ ]:
X = pd.DataFrame(data['New_embeddings'].values.tolist(), index = data['New_embeddings'].index) 

## Модель для площади

In [ ]:
if model_type_area == 'all':
    predictions_area = []
    if size == '-10':
        models = models1
    elif size == '10-100':
        models = models2
    else:
        models = models3
    for model in models:
        model_area = joblib.load(model)
        predictions_area.append(model_area.predict(X))
else:
    # Шаг 2: Загрузка обученной модели
    model_area = joblib.load(model_type_area) 

    # Шаг 3: Применение модели к данным и получение предсказаний
    predictions_area = model_area.predict(X)   

## Модель для задержки

In [ ]:
if model_type_delay == 'all':
    predictions_delay = []
    if size == '-10':
        models = models1
    elif size == '10-100':
        models = models2
    else:
        models = models3
    for model in models:
        model_delay = joblib.load(model)
        predictions_delay.append(model_delay.predict(X))
else:
    # Шаг 2: Загрузка обученной модели
    model_delay = joblib.load(model_type_area) 

    # Шаг 3: Применение модели к данным и получение предсказаний
    predictions_delay = model_delay.predict(X)   

## Формирование результатов

In [ ]:
result = pd.DataFrame()
result['id'] = data['File']
if output_data == 'd':
    result['Delay'] = predictions_delay
elif output_data == 'a':
    result['Area'] = predictions_area
else:
    result['Area'] = predictions_area
    result['Delay'] = predictions_delay

In [ ]:
result.to_csv('submission.csv', index=False)